In [ ]:
train_dir = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training'
test_dir = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test'

In [ ]:
import os
import random

In [ ]:
import tensorflow as tf

IMG_SIZE = (224,224)
BATCH_SIZE = 32

train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                 validation_split=0.2,
                                                                 subset='training',
                                                                 seed=42,
                                                                 label_mode='categorical',
                                                                 image_size=IMG_SIZE)
valid_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                 validation_split=0.2,
                                                                 subset='validation',
                                                                 seed=42,
                                                                 label_mode='categorical',
                                                                 image_size=IMG_SIZE,
                                                                 shuffle=False)
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode='categorical',
                                                                image_size=IMG_SIZE,
                                                                shuffle=False)